## Optimized Evaluation (Faster Version)

If the standard evaluation script takes too long to run, you can use this faster version that limits the number of images used for ROC curves and Grad-CAM visualization.

In [ ]:
# Faster evaluation script that limits the number of images processed
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from PIL import Image

# Function to generate quicker GradCAM visualization
def quick_gradcam(model, img, save_path=None):
    # Find last Conv2D layer
    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer
            break
    
    if last_conv_layer is None:
        print("No Conv2D layer found!")
        return None
    
    # Create a model that outputs both the last conv layer and the final output
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[last_conv_layer.output, model.output]
    )
    
    # Expand dimensions for batch
    img_array = tf.expand_dims(img, axis=0)
    
    # Get gradients
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        class_out = preds[:, 0]
    
    # Gradients of the output with respect to the last conv layer
    grads = tape.gradient(class_out, last_conv_layer_output)
    
    # Pooled gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight the channels by the gradients
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    
    # Apply weights
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
        
    # Average over channels
    heatmap = np.mean(last_conv_layer_output, axis=-1)
    
    # ReLU
    heatmap = np.maximum(heatmap, 0)
    
    # Normalize
    if np.max(heatmap) > 0:
        heatmap = heatmap / np.max(heatmap)
    
    # Resize to original image size
    heatmap = np.uint8(255 * heatmap)
    heatmap = tf.image.resize(
        tf.expand_dims(heatmap, -1),
        (img.shape[0], img.shape[1])
    ).numpy().squeeze()
    
    # Convert image to RGB numpy array
    img_np = img.numpy()
    if img_np.max() <= 1.0:
        img_np = img_np * 255
    img_np = img_np.astype(np.uint8)
    
    # Create heatmap
    cmap = plt.get_cmap('jet')
    colored_heatmap = cmap(heatmap)[:, :, :3]
    colored_heatmap = (colored_heatmap * 255).astype(np.uint8)
    
    # Combine original and heatmap
    superimposed_img = cv2.addWeighted(img_np, 0.6, colored_heatmap, 0.4, 0)
    
    if save_path:
        plt.figure(figsize=(6, 6))
        plt.imshow(superimposed_img)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    
    return superimposed_img

# Load the model
model_path = f"{RESULTS_DIR}/best_model.h5"
if not os.path.exists(model_path):
    model_path = f"{RESULTS_DIR}/model.h5"

print(f"Loading model from: {model_path}")
model = tf.keras.models.load_model(model_path)
model.summary()

# Create a faster dataset with a smaller batch for testing
batch_size = 32
img_size = (224, 224)

# Use processed data structure
test_dir = os.path.join(PROCESSED_DATA_DIR, 'test')
if os.path.exists(test_dir):
    print(f"Loading test data from: {test_dir}")
    
    # Create the test dataset with a smaller batch
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    
    # Limit the evaluation to just a few batches
    max_batches = 5
    print(f"Using only {max_batches} batches for faster evaluation")
    
    # Get predictions for a limited number of batches
    y_true = []
    y_pred = []
    y_scores = []
    
    # Process limited batches for confusion matrix
    print("Generating predictions for confusion matrix...")
    for i, (images, labels) in enumerate(test_generator):
        if i >= max_batches:
            break
            
        # Get predictions
        scores = model.predict(images, verbose=0)
        preds = (scores > 0.5).astype(int)
        
        # Store results
        y_true.extend(labels)
        y_pred.extend(preds.flatten())
        y_scores.extend(scores.flatten())
    
    # Convert to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_scores = np.array(y_scores)
    
    # Calculate metrics
    accuracy = np.mean(y_true == y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    # Add labels
    class_names = ['No Tumor', 'Tumor']
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Add values to the plot
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(os.path.join(RESULTS_DIR, "confusion_matrix_fast.png"))
    plt.show()
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(RESULTS_DIR, "roc_curve_fast.png"))
    plt.show()
    
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    avg_precision = average_precision_score(y_true, y_scores)
    
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2, label=f'Precision-Recall curve (AP = {avg_precision:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="upper right")
    plt.savefig(os.path.join(RESULTS_DIR, "pr_curve_fast.png"))
    plt.show()
    
    # Generate a few GradCAM visualizations (just 2 for speed)
    print("Generating limited GradCAM visualizations...")
    gradcam_dir = os.path.join(RESULTS_DIR, "gradcam_fast")
    os.makedirs(gradcam_dir, exist_ok=True)
    
    # Get a batch of test images
    test_batch = next(test_generator)
    test_images = test_batch[0]
    test_labels = test_batch[1]
    
    # Process just 2 images
    for i in range(2):
        if i < len(test_images):
            img = test_images[i]
            label = test_labels[i]
            pred = model.predict(np.expand_dims(img, 0), verbose=0)[0][0]
            pred_label = 1 if pred > 0.5 else 0
            
            # Create a filename with info
            filename = f"gradcam_fast_{i}_true_{class_names[int(label)]}_pred_{class_names[pred_label]}_{pred:.2f}.png"
            save_path = os.path.join(gradcam_dir, filename)
            
            # Generate and save GradCAM
            try:
                import cv2
                _ = quick_gradcam(model, img, save_path)
            except Exception as e:
                print(f"Error generating GradCAM: {e}")
    
    print("Fast evaluation completed!")
else:
    print("Could not find test directory. Make sure to run preprocessing first.")